# 4. The Standard Model

In this chapter, we'll go through all fields in the standard model, explain their meaning and the default functions that calculate them.

In [1]:
from tripod import Simulation

In [2]:
sim = Simulation()

In [3]:
sim.initialize()

By default, the frame object of `TwoPopPy` consists of four groups for **dust**, **gas**, **grid**, and **stellar** parameters, a field for the **time**, which is the integration variable, an **integrator** object, and a **writer** object.

In [4]:
sim

DustPy
------
    components   : Group (components)
    dust         : Group (Dust quantities)
    gas          : Group (Gas quantities)
    grid         : Group (Grid quantities)
    star         : Group (Stellar quantities)
  -----
    t            : IntVar (Time [s]), Integration variable
  -----
    Integrator   : Integrator (Default integrator)
    Writer       : Writer (HDF5 file format using h5py)

## Dust

In [5]:
sim.dust

Group (Dust quantities)
-----------------------
    backreaction : Group (Backreaction coefficients)
    boundary     : Group (Boundary conditions)
    delta        : Group (Mixing parameters)
    f            : Group (Fudge factors)
    Fi           : Group (Fluxes)
    p            : Group (Probabilities)
    q            : Group (Distribution exponents)
    S            : Group (Sources)
    s            : Group (Characteristic particle sizes)
    v            : Group (Velocities)
  -----
    a            : Field (Particle sizes in cm: [a0, fudge * a1, a1, 0.5 * amax, amax])
    D            : Field (Diffusivity [cm²/s])
    eps          : Field (Dust-to-gas ratio)
    fill         : Field (Filling factor)
    H            : Field (Scale heights [cm])
    m            : Field (Particle mass [g])
    qrec         : Field (reconstructed distribution exponent)
    rho          : Field (Midplane mass density per mass bin [g/cm³])
    rhos         : Field (Solid state density [g/cm³])
  

### `Simulation.dust.backreaction`

In [6]:
sim.dust.backreaction

Group (Backreaction coefficients)
---------------------------------
    A            : Field (Pull factor)
    B            : Field (Push factor)
  -----

The backreaction describes the hydrodynamic influence the dust has on the gas. Numerically, it consists of two fields `A` and `B` of shape `(Simulation.grid.Nr,)` that describe the pull or rather the push the dust excerts on the gas respectively.

The details of this mechanism are described in [Gárate et al. (2019)](https://doi.org/10.3847/1538-4357/aaf4fc).

Backreaction modifies the radial gas velocity as follows

$v_\mathrm{g} = Av_\mathrm{visc} + 2B\eta v_\mathrm{K}$.

In the standard model, we have `A=1` and `B=0` everywhere, i.e. backreactions are not active.

In [7]:
sim.dust.backreaction.A

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]

In [8]:
sim.dust.backreaction.B

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]

Depending on the type of backreaction that we want to model, we have to provide functions for `A` and `B`. Please also have a look at Appendix A of [Gárate et al. (2019)](https://doi.org/10.3847/1538-4357/aaf4fc) for examples.

### `Simulation.dust.boundary`

In [9]:
sim.dust.boundary

Group (Boundary conditions)
---------------------------
    inner        : Boundary
    outer        : Boundary
  -----

By default, the inner dust boundary is set to constant gradient and the outer boundary to floor value.

In [10]:
sim.dust.boundary.inner

Constant gradient

In [11]:
sim.dust.boundary.outer

Value

The outer boundary therefore stores an array with the floor values of all particle masses.

The boundary conditions can be changed via `setcondition()`.

In [12]:
help(sim.dust.boundary.outer.setcondition)

Help on method setcondition in module dustpy.utils.boundary:

setcondition(condition, value=None) method of dustpy.utils.boundary.Boundary instance
    Function to set boundary condition.

    Parameters
    ----------
    condition : string
        Type of boundary conditon:
            - "const_grad" : constant gradient
            - "const_pow" : constant power law
            - "const_val" : constant value
            - "val" : custom value
            - "grad" : custom gradient
            - "pow" : custom power law with set exponent
            - None : Don't impose boundary condition (default)
    value : float or array, optional, default : None
        Value if needed for boundary condition



### `Simulation.dust.delta`

In [13]:
sim.dust.delta

Group (Mixing parameters)
-------------------------
    rad          : Field (Radial mixing parameter)
    turb         : Field (Turbulent mixing parameter)
    vert         : Field (Vertical mixing parameter)
  -----

The $\delta$ parameters control the mixing of dust particles along vertical and radial directions and turbulent mixing. We can look at them as similar to the turbulent $\alpha$ parameter for the gas. By default, they will also have the same value as $\alpha$ as given by `Simulation.ini.gas.alpha`.

#### `Simulation.dust.delta.rad`

In [14]:
sim.dust.delta.rad

[0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001]

$\delta_\mathrm{rad}$ will be used to calculate the radial dust diffusion.

#### `Simulation.dust.delta.turb`

In [15]:
sim.dust.delta.turb

[0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001]

$\delta_\mathrm{turb}$ will be used to calculate the turbulent collision velocities of the dust particles.

#### `Simulation.dust.delta.vert`

In [16]:
sim.dust.delta.vert

[0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001]

$\delta_\mathrm{vert}$ will be used to calculate the vertical dust scale heights.

### `Simulation.dust.Fi`

In [17]:
sim.dust.Fi

Group (Fluxes)
--------------
    adv          : Field (Advective flux [g/cm/s])
    diff         : Field (Diffusive flux [g/cm/s])
    tot          : Field (Total flux [g/cm/s])
  -----

This is a group of fluxes through the radial grid interfaces for every particle mass of shape `(Simulation.grid.Nr+1, 2)`.

**Attention:** When using implicit dust integration, the interface fluxes are calculated in retrospect after the new dust surface density was found. Changing any `Simulation.dust.Fi` does not influence the simulation. It is only given for data analysis.

#### `Simulation.dust.Fi.adv`

This is the advective flux calculated by $F_\mathrm{adv} = v_\mathrm{d}\Sigma_\mathrm{d}$.

**Attention:** When using implicit dust integration, the advective interface fluxes are calculated in retrospect after the new dust surface density was found. Changing `Simulation.dust.Fi.adv` does not influence the simulation. It is only given for data analysis.

#### `Simulation.dust.Fi.diff`

This is the diffusive flux calculated by $F_\mathrm{diff} = -D\Sigma_\mathrm{g}\nabla\frac{\Sigma_\mathrm{d}}{\Sigma_\mathrm{gas}}$ for every particle species separately. The diffusive fluxes at the grid boundaries are set to zero to avoid instabilities.

**Attention:** When using implicit dust integration, the diffusive interface fluxes are calculated in retrospect after the new dust surface density was found. Changing `Simulation.dust.Fi.diff` does not influence the simulation. It is only given for data analysis.

#### `Simulation.dust.Fi.tot`

This is the total flux through the radial grid interfaces $F_\mathrm{tot} = F_\mathrm{adv} + F_\mathrm{diff}$.

**Attention:** When using implicit dust integration, the total interface fluxes are calculated in retrospect after the new dust surface density was found. Changing `Simulation.dust.Fi.tot` does not influence the simulation. It is only given for data analysis.

### `Simulation.dust.p`

In [18]:
sim.dust.p

Group (Probabilities)
---------------------
    driftfrag    : Field (Transition function from drift to turbulence)
    frag         : Field (Fragmentation probability)
    fragtrans    : Field (transition probability between fragmentation regimes)
    stick        : Field (Sticking probability)
  -----

These are the fragmentation and sticking probability of certain particle collisions.  
Their shape is `(Simulation.grid.Nr, 4, 4)`.

Collisions between particles of sizes $0.5*<a>_m$ (`i=3`) and $<a>_m$ (`j=2`) are representative for the entire dust distribution. The characteristic fragmentation probability at radial grid cell `ir=15` is therefore given by

In [19]:
ir = 15
sim.dust.p.frag[ir]

np.float64(1.6517614387090067e-11)

#### `Simulation.dust.p.frag`

`TwoPopPy` assumes that the relative collision velocities follow the Maxwell-Boltzmann distribution

$f\left(\Delta v\right) = \sqrt{\frac{54}{\pi}}\frac{\Delta v^{2}}{v_\mathrm{rms}^3} \exp \left[-\frac{3}{2} \frac{\Delta v^2}{v_\mathrm{rms}^2} \right]$.

The collision kernel for fragmentation is then given by

$K_\mathrm{f} = \int\limits_{v_\mathrm{frag}}^\infty \sigma_\mathrm{geo} \Delta v f\left(\Delta v\right) \mathrm{d}\Delta v$

by counting all collisions above the fragmentation velocity.

The fragmentation probability is then given by

$p_\mathrm{f} = \frac{K_\mathrm{f}}{\sigma_\mathrm{geo} \Delta \bar{v}}$

with the mean velocity of the Maxwell-Boltzmann distribution

$\Delta \bar{v} = \sqrt{\frac{8\pi}{2}}v_\mathrm{rms}$.

The root-mean-square velocities $v_\mathrm{rms}$ are stored in `Simulation.dust.v.rel`.

#### `Simulation.dust.p.stick`

This is the sticking probability given by $p_\mathrm{s} = 1 - p_\mathrm{f}$.

Bouncing can be easily implemented if $0 \leq p_\mathrm{f} + p_\mathrm{s} < 1$.

### `Simulation.dust.S`

In [20]:
sim.dust.S

Group (Sources)
---------------
    coag         : Field (Coagulation sources [g/cm²/s])
    compo        : Field (Sources due to composition changes [g/cm²/s])
    ext          : Field (External sources [g/cm²/s])
    hyd          : Field (Hydrodynamic sources [g/cm²/s])
    smax_hyd     : Field (Total sources [g/cm²/s])
    tot          : Field (Total sources [g/cm²/s])
  -----

These are the source terms of the dust of shape `(Simulation.grid.Nr, 2)` used to integrate the time evolution of the dust.

#### `Simulation.dust.S.coag`

The source terms from dust coagulation given by 

$S_{\mathrm{coag},\,0 \rightarrow 1} = \dot{\Sigma}_{0 \rightarrow 1} - \dot{\Sigma}_{1 \rightarrow 0} =  \frac{\Sigma_0 \Sigma_1 \sigma_{01} \Delta v_{01}}{m_1 \sqrt{2 \pi \left(H_0^2 + H_1^2\right)}} - \frac{\Sigma_1^2 \sigma_{11} \Delta v_{11}}{m_1 2 \sqrt{\pi} H_1} \mathcal{F} = - S_{\mathrm{coag},\,1 \rightarrow 0}$

with $\mathcal{F} = \sqrt{\frac{2H_1^2}{H_0^2 + H_1^2}} \frac{\sigma_{01}\Delta v_{01}}{\sigma_{11}\Delta v_{11}}\left(\frac{s_{max}}{s_{min}}\right)^{-\left(\xi'+4\right)}$ and $\xi' = \Pi_{frag}\xi_{frag} + \Pi_{stick}\xi_{stick}$

#### `Simulation.dust.S.ext`

External source terms. These are by default set to zero, i.e. no external sources.

In [21]:
sim.dust.S.ext

[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]

#### `Simulation.dust.S.hyd`

These are the hydrodynamic source terms. At grid cell $i$ they are calculated by

$S_{\mathrm{hyd},\,i} = 2\frac{\left( r_{i-\frac{1}{2}}F_{i-\frac{1}{2}}-r_{i+\frac{1}{2}}F_{i+\frac{1}{2}} \right)}{r_{i+\frac{1}{2}}^2 - r_{i-\frac{1}{2}}^2}$.

**Attention:** When using implicit dust integration, the hydrodynamic source terms are calculated in retrospect after the new dust surface density was found. Changing `Simulation.dust.S.hyd` does not influence the simulation. It is only given for data analysis.

#### `Simulation.dust.S.tot`

These are the total source terms given by $S_\mathrm{tot} = S_\mathrm{coag} + S_\mathrm{ext} + S_\mathrm{hyd}$.

### `Simulation.dust.s`

In [22]:
sim.dust.s

Group (Characteristic particle sizes)
-------------------------------------
    boundary     : Group (boundary conditions of smax)
  -----
    lim          : Field (Limiting size for shrinking)
    max          : Field (Maximum particle size)
    min          : Field (Minimum particle size)
    sdot_coag    : Field (coagulation source term for amax [cm/s])
    sdot_shrink  : Field (shrinkage source term for amax [cm²/s])
  -----

These are specific particle sizes of shape `(Simulation.grid.Nr)` characterizing the dust distribution.

#### `Simulation.dust.s.max`

This is the maximum particle size of the dust distribution. By default, it is initialized as `Simulation.ini.dust.aIniMax`.

In [23]:
sim.dust.s.max

[1.00000000e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.000000

Over the course of the simulation, it is evolved in time according to the source term

$\dot{s}_{max} = \frac{\rho_d}{\rho_s}\Delta v \left[\frac{1-\left(\frac{v_{frag}}{\Delta v}\right)^s}{1+\left(\frac{v_{frag}}{\Delta v}\right)^s}\right]$

describing its growth and decline due to sticking and fragmentation.

#### `Simulation.dust.s.min`

This is the minimum particle size of the dust distribution, which remains constant over time. It is by default set to $0.1 *$ `Simulation.ini.dust.aIniMax`.

In [24]:
sim.dust.s.min

[5.22875516e-05 5.22875516e-05 5.22875516e-05 5.22875516e-05
 5.22875516e-05 5.22875516e-05 5.22875516e-05 5.22875516e-05
 5.22875516e-05 5.22875516e-05 5.22875516e-05 5.22875516e-05
 5.22875516e-05 5.22875516e-05 5.22875516e-05 5.22875516e-05
 5.22875516e-05 5.22875516e-05 5.22875516e-05 5.22875516e-05
 5.22875516e-05 5.22875516e-05 5.22875516e-05 5.22875516e-05
 5.22875516e-05 5.22875516e-05 5.22875516e-05 5.22875516e-05
 5.22875516e-05 5.22875516e-05 5.22875516e-05 5.22875516e-05
 5.22875516e-05 5.22875516e-05 5.22875516e-05 5.22875516e-05
 5.22875516e-05 5.22875516e-05 5.22875516e-05 5.22875516e-05
 5.22875516e-05 5.22875516e-05 5.22875516e-05 5.22875516e-05
 5.22875516e-05 5.22875516e-05 5.22875516e-05 5.22875516e-05
 5.22875516e-05 5.22875516e-05 5.22875516e-05 5.22875516e-05
 5.22875516e-05 5.22875516e-05 5.22875516e-05 5.22875516e-05
 5.22875516e-05 5.22875516e-05 5.22875516e-05 5.22875516e-05
 5.22875516e-05 5.22875516e-05 5.22875516e-05 5.22875516e-05
 5.22875516e-05 5.228755

### `Simulation.dust.v`

In [25]:
sim.dust.v

Group (Velocities)
------------------
    rel          : Group (Relative velocities)
  -----
    driftmax     : Field (Maximum drift velocity [cm/s])
    frag         : Field (Fragmentation velocity [cm/s])
    rad          : Field (Radial velocity [cm/s])
    rad_flux     : Field (Radial velocity modified to claulate the proper flux[cm/s])
  -----

These are some dust related velocities the simulation needs for execution. 

#### `Simulation.dust.v.rel`

In [26]:
sim.dust.v.rel

Group (Relative velocities)
---------------------------
    azi          : Field (Relative azimuthal velocity [cm/s])
    brown        : Field (Relative Brownian motion velocity [cm/s])
    rad          : Field (Relative radial velocity [cm/s])
    tot          : Field (Total relative velocity [cm/s])
    turb         : Field (Relative turbulent velocity [cm/s])
    vert         : Field (Relative vertical settling velocity [cm/s])
  -----

These are the different sources of relative particle collision velocities used by the coagulation algorithm.  
They are used for two different reasons:
1. To calculated the collision rates
2. To determine the outcome of a collision

#### `Simulation.dust.v.rel.azi`

Relative collision velocity caused by a differential azimuthal drift of particles with different Stokes numbers calculated by

$v_{\mathrm{rel},\,\mathrm{azi}} = \left| v_{\mathrm{drift},\,\mathrm{max}} \cdot \left( \frac{1}{1+\mathrm{St}_i^2} - \frac{1}{1+\mathrm{St}_j^2} \right) \right|$.

##### `Simulation.dust.v.brown`

Relative collision velocity of particles caused by Brownian motion calculated with

$v_{\mathrm{rel},\,\mathrm{brown}} = \sqrt{ \frac{8k_\mathrm{B}T\left(m_i + m_j \right)}{\pi m_i m_j} }$.

Since this expression is diverging for very small particle masses, the relative velocity is capped to a maximum value of the sound speed $c_\mathrm{s}$. For very small particles this can still be larger than the fragmentation velocity and can cause unwanted fragmentation in the simple coagulation algorithm implemented in the default model.

##### `Simulation.dust.v.rel.rad`

Relative collision velocity caused by differential radial drift.

$v_{\mathrm{rel},\,\mathrm{rad}} = \left| v_{\mathrm{rad},\,i} - v_{\mathrm{rad},\,j} \right|$.

##### `Simulation.dust.v.rel.tot`

Total relative velocities calculated by using the root mean square of all individual velocity sources.

$v_{\mathrm{rel},\,\mathrm{tot}} = \sqrt{v_{\mathrm{rel},\,\mathrm{azi}}^2 + v_{\mathrm{rel},\,\mathrm{brown}}^2 + v_{\mathrm{rel},\,\mathrm{rad}}^2 + v_{\mathrm{rel},\,\mathrm{turb}}^2 + v_{\mathrm{rel},\,\mathrm{vert}}^2}$.

##### `Simulation.dust.v.rel.turb`

Relative velocities caused by turbulent motion of the particles. This follows the prescription of [Ormel & Cuzzi (2007)](https://doi.org/10.1051/0004-6361:20066899).  
It uses `Simulation.dust.delta.turb` instead of `Simulation.gas.alpha` to calculate the velocities.

##### `Simulation.dust.v.rel.vert`

Relative collision velocities caused by differential vertical settling of particles.

$v_{\mathrm{rel},\,\mathrm{vert}} = \left| h_i \min \left( \mathrm{St}_i,\,\frac{1}{2}\right) - h_j \min \left( \mathrm{St}_j,\,\frac{1}{2}\right) \right| \cdot \Omega_\mathrm{K}$.

This prescription is taken from [Birnstiel et al. (2010)](https://doi.org/10.1051/0004-6361/200913731) and follows [Dullemond & Dominik (2004)](https://doi.org/10.1051/0004-6361:20040284).

#### `Simulation.dust.v.driftmax`

This is the maximum drift velocity a particle of $\mathrm{St} = 1$ can have.

$v_{\mathrm{drift},\,\mathrm{max}} = \frac{1}{2} B v_\mathrm{visc} - A \eta v_\mathrm{K}$.

See `Simulation.dust.backreaction` for details.

#### `Simulation.dust.v.frag`

Fragmentation velocities of shape `(Simulation.grid.Nr)`. By default this is set by the value of `Simulation.ini.dust.vfrag`.

In [27]:
sim.dust.v.frag

[100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100.]

#### `Simulation.dust.v.rad`

Radial velocities of the dust particles.

$v_\mathrm{rad} = \left( v_\mathrm{g} + 2 v_{\mathrm{drift},\,\mathrm{max}}\mathrm{St} \right) \frac{1}{\mathrm{St}^2+1}$

### `Simulation.dust.xi`

In [28]:
sim.dust.qrec

[-3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5
 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5
 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5
 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5
 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5
 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -4.  -4.  -4.  -4.  -4.  -4.
 -4.  -4.  -4.  -4.  -4.  -4.  -4.  -4.  -4.  -4.  -4.  -4.  -4.  -4.
 -4.  -4. ]

These are exponents of the particle size distribution $\sigma\left(a\right) \propto a^\xi$ of shape `(Simulation.grid.Nr)`.

#### `Simulation.dust.xi.calc`

This is the exponent of the current particle size distribution calculated via

$\xi = \frac{\log\left(\Sigma_1/\Sigma_0\right)}{\log\left(s_{max}/s_{int}\right)} - 4$

#### `Simulation.dust.xi.frag`

This is the exponent of the particle size distribution in the fragmentation limited case. It is initialized as negative `Simulation.ini.distExp` by default.

In [31]:
sim.dust.q.frag

[-3.75       -3.75       -3.75       -3.75       -3.75       -3.75
 -3.75       -3.75       -3.75       -3.75       -3.75       -3.75
 -3.75       -3.75       -3.75       -3.75       -3.75       -3.75
 -3.75       -3.75       -3.75       -3.75       -3.75       -3.75
 -3.75       -3.75       -3.75       -3.75       -3.75       -3.75
 -3.75       -3.75       -3.75       -3.75       -3.75       -3.75
 -3.75       -3.75       -3.75       -3.75       -3.75       -3.75
 -3.75       -3.75       -3.75       -3.75       -3.75       -3.75
 -3.75       -3.75       -3.75       -3.75       -3.75       -3.75
 -3.75       -3.75       -3.75       -3.75       -3.75       -3.75
 -3.75       -3.75       -3.74999999 -3.74999999 -3.74999999 -3.74999998
 -3.74999998 -3.74999996 -3.74999995 -3.74999992 -3.74999988 -3.74999981
 -3.7499997  -3.7499995  -3.74999916 -3.74999855 -3.74999742 -3.74999526
 -3.74999659 -3.74999328 -3.74998623 -3.74997062 -3.74993443 -3.74984661
 -3.74962289 -3.74908028 -3.74964369 -

#### `Simulation.dust.xi.stick`

This is the exponent of the particle size distribution in the drift limited case. By default, it is set to negative `Simulation.ini.distExp` $+ 1$.

In [34]:
sim.dust.q.sweep

-3.0

### `Simulation.dust.a`

Particle radii of shape `(Simulation.grid.Nr, 5)` calculated with

$a_0 = \left\{ \begin{aligned} \frac{s_{int}s_{min}}{s_{int}-s_{min}} \log\left(s_{int}/s_{min}\right) \textrm{ for }\xi = -5 \\ \frac{s_{int}-s_{min}}{\log\left(s_{int}/s_{min}\right)} \textrm{ for }\xi = -4 \\ \frac{\xi+4}{\xi+5} \frac{s_{int}^{\xi+5}-s_{min}^{\xi+5}}{s_{int}^{\xi+4}-s_{min}^{\xi+4}} \textrm{ for }\xi \neq -4, -5\end{aligned} \right.$

$a_1 = \left\{ \begin{aligned} \frac{s_{max}s_{int}}{s_{max}-s_{int}} \log\left(s_{max}/s_{int}\right) \textrm{ for }\xi = -5 \\ \frac{s_{max}-s_{int}}{\log\left(s_{max}/s_{int}\right)} \textrm{ for }\xi = -4 \\ \frac{\xi+4}{\xi+5} \frac{s_{max}^{\xi+5}-s_{int}^{\xi+5}}{s_{max}^{\xi+4}-s_{int}^{\xi+4}} \textrm{ for }\xi \neq -4, -5\end{aligned} \right.$

$<a>_m = \left\{ \begin{aligned} \frac{s_{max}s_{min}}{s_{max}-s_{min}} \log\left(s_{max}/s_{min}\right) \textrm{ for }\xi = -5 \\ \frac{s_{max}-s_{min}}{\log\left(s_{max}/s_{min}\right)} \textrm{ for }\xi = -4 \\ \frac{\xi+4}{\xi+5} \frac{s_{max}^{\xi+5}-s_{min}^{\xi+5}}{s_{max}^{\xi+4}-s_{min}^{\xi+4}} \textrm{ for }\xi \neq -4, -5\end{aligned} \right.$

### `Simulation.dust.D`

Dust diffusivity for every particle species of shape `(Simulation.grid.Nr, 4)` calculated with

$D = \frac{\delta_\mathrm{rad}\,c_\mathrm{s}^2}{\Omega_\mathrm{K}\left( 1 + \mathrm{St}^2 \right)}$.

### `Simulation.dust.eps`

This is the vertically integrated dust-to-gas ratio of shape `(Simulation.grid.Nr)`. In the literature this is also often refered to as metallicity $z$. It is calculated via

$\epsilon = \frac{\sum\limits_i \Sigma_{\mathrm{d},\,i}}{\Sigma_\mathrm{g}}$

### `Simulation.dust.fill`

This describes the filling factor of the dust aggregates. By default this is 1.

In [35]:
sim.dust.fill

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1.

### `Simulation.dust.H`

These are the dust scale heights of shape `(Simulation.grid.Nr, 4)` calculated with the prescription of [Dubrulle et al. (1995)](https://doi.org/10.1006/icar.1995.1058)

$H_i = H_\mathrm{P} \cdot \sqrt{\frac{\delta_\mathrm{vert}}{\delta_\mathrm{vert}+\mathrm{St}_i}}$

It is limited to a maximum of the pressure scale height $H_\mathrm{P}$. It uses `Simulation.dust.delta.vert` as vertical mixing parameter instead of `Simulation.gas.alpha`.

### `Simulation.dust.m`

Particle masses of shape `(Simulation.grid.Nr, 4)` calculated with

$m = \frac{4}{3} \pi \rho_s a^3$

### `Simulation.dust.rho`

Midplane mass densities of the dust.

$\rho = \frac{\Sigma}{\sqrt{2\pi}H}$

### `Simulation.dust.rhos`

Solid state density of the particle material. This is initially set to `Simulation.ini.dust.rhoMonomer`.

In [36]:
sim.dust.rhos

[[1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.6

To calculate the mass density of a dust aggregate do

In [37]:
sim.dust.fill * sim.dust.rhos

[[1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.6

This is the density that is used to calculate the particle size.

### `Simulation.dust.Sigma`

This is the dust surface density of the small and large particle species of shape `(Simulation.grid.Nr, 2)`. It is the quantity that is integrated to solve for dust evolution.

The surface densities are integrated over the mass bin, so a numerical summation over the mass dimension returns the total dust surface density.

$\Sigma_\mathrm{d} = \sum\limits_i \Sigma_{\mathrm{d},\,i}$

In [38]:
sim.dust.Sigma.sum(-1)

[1.11815368e+01 1.04232155e+01 9.71465059e+00 9.05339692e+00
 8.43629793e+00 7.86040780e+00 7.32297742e+00 6.82144128e+00
 6.35340519e+00 5.91663488e+00 5.50904531e+00 5.12869076e+00
 4.77375546e+00 4.44254502e+00 4.13347826e+00 3.84507969e+00
 3.57597248e+00 3.32487186e+00 3.09057900e+00 2.87197527e+00
 2.66801690e+00 2.47773001e+00 2.30020595e+00 2.13459693e+00
 1.98011200e+00 1.83601326e+00 1.70161231e+00 1.57626698e+00
 1.45937826e+00 1.35038739e+00 1.24877321e+00 1.15404966e+00
 1.06576343e+00 9.83491798e-01 9.06840556e-01 8.35442144e-01
 7.68953850e-01 7.07056150e-01 6.49451154e-01 5.95861146e-01
 5.46027226e-01 4.99708026e-01 4.56678518e-01 4.16728886e-01
 3.79663471e-01 3.45299782e-01 3.13467557e-01 2.84007884e-01
 2.56772375e-01 2.31622377e-01 2.08428234e-01 1.87068586e-01
 1.67429708e-01 1.49404882e-01 1.32893806e-01 1.17802035e-01
 1.04040457e-01 9.15247991e-02 8.01751688e-02 6.99156307e-02
 6.06738170e-02 5.23805770e-02 4.49696665e-02 3.83774784e-02
 3.25428186e-02 2.740672

If you want to plot the dust density distribution, you might want to convert the field into a quantity that does not depend on the mass bin width. The plotting script integrated in `TwoPopPy` is plotting a grid independent density distribution. See the first chapter for details on this.

### `Simulation.dust.SigmaFloor`

This is the floor value for the dust surface densities. Mass bins that are below their respective floor value will not contribute to coagulation. By default, the floor value is the density that corresponds to one physical particle of that mass distributed over an annulus at that radial grid location.

$\Sigma_{\mathrm{d},\,\mathrm{floor}} = \frac{m}{A_\mathrm{annulus}}$

Densities below the floor value therefore correspond to fewer that one physical particle in that radial grid cell.

### `Simulation.dust.St`

Stokes numbers of the particles. By default the Epstein and the Stokes I drag regime are considered.

$\mathrm{St} =
\begin{cases}
\frac{\pi}{2} \frac{a\rho}{\Sigma_\mathrm{g}}, & \text{if } a < \frac{9}{4} \lambda_\mathrm{mfp}\\
\frac{2\pi}{9} \frac{a^2 \rho}{\lambda_\mathrm{mfp} \Sigma_\mathrm{g}}, & \text{else}
\end{cases}$

### Update order

The update order of the dust quantities in the standard model is set to

In [39]:
sim.dust.updateorder

['delta',
 'rhos',
 'fill',
 'backreaction',
 'f',
 'qrec',
 'a',
 'm',
 'St',
 'H',
 'rho',
 'D',
 'eps',
 'v',
 'p',
 'q',
 'SigmaFloor',
 'S']

In [40]:
sim.dust.backreaction.updateorder

['A', 'B']

In [41]:
sim.dust.delta.updateorder

['rad', 'turb', 'vert']

In [42]:
sim.dust.Fi.updateorder

['adv', 'diff', 'tot']

In [43]:
sim.dust.p.updateorder

['frag', 'stick', 'fragtrans', 'driftfrag']

In [44]:
sim.dust.S.updateorder

['ext', 'tot', 'smax_hyd']

In [45]:
sim.dust.s.updateorder

In [46]:
sim.dust.v.updateorder

['frag', 'driftmax', 'rel']

In [47]:
sim.dust.v.rel.updateorder

['azi', 'brown', 'rad', 'turb', 'vert', 'tot']

In [49]:
sim.dust.q.updateorder

['frag', 'eff']

**Note**: The quantities that are excluded are calculated in the finalization step of the implicit integrator from the new values of the dust surface densities.

## Gas

In [50]:
sim.gas

Group (Gas quantities)
----------------------
    boundary     : Group (Boundary conditions)
    S            : Group (Source terms)
    torque       : Group (Torque parameters)
    v            : Group (Velocities)
  -----
    alpha        : Field (Turbulent alpha parameter)
    cs           : Field (Isothermal sound speed [cm/s])
    eta          : Field (Pressure gradient parameter)
    Fi           : Field (Gas flux interfaces [g/cm/s])
    Hp           : Field (Pressure scale height [cm])
    mfp          : Field (Midplane mean free path [cm])
    mu           : Field (Mean molecular weight [g])
    n            : Field (Miplane number density [1/cm³])
    nu           : Field (Kinematic viscosity [cm²/s])
    P            : Field (Midplane pressure [g/cm/s²])
    rho          : Field (Miplane mass density [g/cm³])
    Sigma        : Field (Surface density [g/cm²])
    SigmaFloor   : Field (Floor value of surface density [g/cm²])
    T            : Field (Temperature [K])
  -----

### `Simulation.gas.boundary`

In [51]:
sim.gas.boundary

Group (Boundary conditions)
---------------------------
    inner        : Boundary
    outer        : Boundary
  -----

These are the boundary conditions of the gas. By default the inner boundary is set to constant gradient and the outer boundary to floor value.

In [52]:
sim.gas.boundary.inner

Constant gradient

In [53]:
sim.gas.boundary.outer

Value

The boundary conditions can be modified with `setcondition`.

In [54]:
help(sim.gas.boundary.inner.setcondition)

Help on method setcondition in module dustpy.utils.boundary:

setcondition(condition, value=None) method of dustpy.utils.boundary.Boundary instance
    Function to set boundary condition.

    Parameters
    ----------
    condition : string
        Type of boundary conditon:
            - "const_grad" : constant gradient
            - "const_pow" : constant power law
            - "const_val" : constant value
            - "val" : custom value
            - "grad" : custom gradient
            - "pow" : custom power law with set exponent
            - None : Don't impose boundary condition (default)
    value : float or array, optional, default : None
        Value if needed for boundary condition



If the gas surface density follows a power law $\propto R^{-1}$ the constant gradient boundary condition should work fine. Other values can lead to deviations at the inner boundary. See the chapter about gas evolution tests for details.

### `Simulation.gas.S`

In [55]:
sim.gas.S

Group (Source terms)
--------------------
    ext          : Field (External sources [g/cm²/s])
    hyd          : Field (Hydrodynamic sources [g/cm²/s])
    tot          : Field (Total sources [g/cm²/s])
  -----

These are the source terms of the gas.

#### `Simulation.gas.S.ext`

These are the external source terms for gas evolution, e.g. infall. By default these are set to zero.

In [56]:
sim.gas.S.ext

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]

#### `Simulation.gas.S.hyd`

These are the hydrodynamic source terms of the gas evolution.  
**Attention:** Since the gas evolution is calculated implicitly, the hydrodynamic source terms are calculated in retrospect after the new gas surface density was found. Changing `Simulation.gas.S.hyd` does not influence the simulation. It is only given for data analysis.

#### `Simulation.gas.S.tot`

These are the total source terms of gas evolution.

$S_\mathrm{tot} = S_\mathrm{ext} + S_\mathrm{hyd}$

**Attention:** Since the gas evolution is calculated implicitly, the total source terms are calculated in retrospect after the new gas surface density was found. Changing `Simulation.gas.S.tot` does not influence the simulation. It is only given for data analysis.

### `Simulation.gas.v`

In [57]:
sim.gas.v

Group (Velocities)
------------------
    rad          : Field (Radial velocity [cm/s])
    visc         : Field (Viscous accretion velocity [cm/s])
  -----

These are velocities that are relevant for the gas evolution.  
**Attention:** Since the gas evolution is calculated implicitly, the velocities are calculated in retrospect after the new gas surface density was found. Changing anything in `Simulation.gas.v` does not influence the simulation. It is only given for data analysis.

#### `Simulation.gas.v.rad`

This is the radial gas velocity. It is given by

$v_\mathrm{g} = Av_\mathrm{visc} + 2B\eta v_\mathrm{K}$.

See `Simulation.dust.backreaction` for details. If backreaction is turned off, i.e., $A=1$ and $B=0$, the radial velocity is identical to the viscous velocity.  
**Attention:** Since the gas evolution is calculated implicitly, the velocities are calculated in retrospect after the new gas surface density was found. Changing anything in `Simulation.gas.v.rad` does not influence the simulation. It is only given for data analysis.

#### `Simulation.gas.v.visc`

This is the radial viscous gas velocity

$v_\mathrm{visc} = -\frac{3}{\Sigma_\mathrm{g}\sqrt{R}} \frac{\partial}{\partial R} \left( \Sigma_\mathrm{g} \nu \sqrt{R} \right)$

**Attention:** Since the gas evolution is calculated implicitly, the velocities are calculated in retrospect after the new gas surface density was found. Changing anything in `Simulation.gas.v.visc` does not influence the simulation. It is only given for data analysis.

### `Simulation.gas.alpha`

This is the turbulent viscosity parameter according [Shakura & Sunyaev (1973)](https://ui.adsabs.harvard.edu/abs/1973A%26A....24..337S/abstract). It is initially set to the value in `Simulation.ini.gas.alpha`.

In [58]:
sim.gas.alpha

[0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001]

### `Simulation.gas.cs`

This is the adiabatic sound speed in the midplane of the disk

$c_\mathrm{s} = \sqrt{\frac{\gamma k_\mathrm{B} T}{\mu}}$.

For isothermal simulations set $\gamma=1$.

### `Simulation.gas.eta`

This is the midplane pressure gradient parameter $\eta$ given by

$\eta = -\frac{1}{2} \left( \frac{H_\mathrm{P}}{r} \right)^2 \frac{\partial \log P}{\partial \log r}$

It describes the degree of "sub-Keplerity" of the disk

$v_\phi^2 = \left( 1-2\eta \right) v_\mathrm{K}^2$.

### `Simulation.gas.Fi`

These are the mass fluxes of gas through the grid cell interfaces.  
**Attention:** Since the gas evolution is calculated implicitly, the fluxes are calculated in retrospect after the new gas surface density was found. Changing anything in `Simulation.gas.Fi` does not influence the simulation. It is only given for data analysis.

### `Simulation.gas.Hp`

Pressure scale height of the gas given by the ratio of the isothermal sound speed to the Keplerian frequency.

$H_\mathrm{P} = \frac{c_{\mathrm{s},\,\mathrm{iso}}}{\Omega_\mathrm{K}}$.

### `Simulation.gas.mfp`

Mean free path of the gas in the midplane of the disk

$\lambda_\mathrm{mfp} = \frac{1}{\sqrt{2}\,n\,\sigma_\mathrm{H_2}}$

### `Simulation.gas.mu`

Mean molecular weight of the gas. This is initially set by the value given in `Simulation.ini.gas.mu` and is equal to $2.3\,m_\mathrm{P}$.

In [60]:
sim.gas.mu

[3.84968137e-24 3.84968137e-24 3.84968137e-24 3.84968137e-24
 3.84968137e-24 3.84968137e-24 3.84968137e-24 3.84968137e-24
 3.84968137e-24 3.84968137e-24 3.84968137e-24 3.84968137e-24
 3.84968137e-24 3.84968137e-24 3.84968137e-24 3.84968137e-24
 3.84968137e-24 3.84968137e-24 3.84968137e-24 3.84968137e-24
 3.84968137e-24 3.84968137e-24 3.84968137e-24 3.84968137e-24
 3.84968137e-24 3.84968137e-24 3.84968137e-24 3.84968137e-24
 3.84968137e-24 3.84968137e-24 3.84968137e-24 3.84968137e-24
 3.84968137e-24 3.84968137e-24 3.84968137e-24 3.84968137e-24
 3.84968137e-24 3.84968137e-24 3.84968137e-24 3.84968137e-24
 3.84968137e-24 3.84968137e-24 3.84968137e-24 3.84968137e-24
 3.84968137e-24 3.84968137e-24 3.84968137e-24 3.84968137e-24
 3.84968137e-24 3.84968137e-24 3.84968137e-24 3.84968137e-24
 3.84968137e-24 3.84968137e-24 3.84968137e-24 3.84968137e-24
 3.84968137e-24 3.84968137e-24 3.84968137e-24 3.84968137e-24
 3.84968137e-24 3.84968137e-24 3.84968137e-24 3.84968137e-24
 3.84968137e-24 3.849681

### `Simulation.gas.n`

Midplane number density of the gas given by

$n = \frac{\rho}{\mu}$

### `Simulation.gas.nu`

Kinematic viscosity of the gas given by

$\nu = \alpha c_\mathrm{s} H_\mathrm{P}$.

### `Simulation.gas.P`

Midplane gas pressure given by

$P = \frac{\rho\,c_\mathrm{s}^2}{\gamma}$.

### `Simulation.gas.rho`

Midplane gas mass density given by

$\rho = \frac{\Sigma_\mathrm{g}}{\sqrt{2\pi}H_\mathrm{P}}$.

### `Simulation.gas.Sigma`

Gas surface density. This is the quantity that is integrated with an implicit Euler first-order scheme.

### `Simulation.gas.SigmaFloor`

This is the floor value of the gas surface density. By default it is $10^{-100}$.

In [61]:
sim.gas.SigmaFloor

[1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100
 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100
 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100
 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100
 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100
 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100
 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100
 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100
 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100
 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100
 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100
 1.e-100]

If the gas surface density is at any point below it's floor value it will be automatically set to the floor value at the end of a time step.

### `Simulation.gas.T`

This is the midplane gas temperature. It is calculated by assuming a passively irradiated disk with a constant irradiation angle of $0.05$.

$T\left( r \right) = \sqrt[4]{\frac{0.05\,L_*}{4\,\pi\,r^2\,\sigma_\mathrm{SB}}}$

### Update order

The update order of the gas quantities in the standard model is set to

In [62]:
sim.gas.updateorder

['Sigma',
 'mu',
 'T',
 'alpha',
 'cs',
 'Hp',
 'nu',
 'rho',
 'n',
 'mfp',
 'P',
 'eta',
 'torque',
 'S']

In [63]:
sim.gas.S.updateorder

['ext', 'tot']

In [64]:
sim.gas.v.updateorder

['visc', 'rad']

**Note**: The quantities that are excluded are calculated in the finalization step of the implicit integrator from the new values of the surface densities.

## Grid

In [65]:
sim.grid

Group (Grid quantities)
-----------------------
    A            : Field (Radial grid annulus area [cm²]), constant
    Nr           : Field (# of radial grid cells), constant
    OmegaK       : Field (Keplerian frequency [1/s])
    r            : Field (Radial grid cell centers [cm]), constant
    ri           : Field (Radial grid cell interfaces [cm]), constant
  -----

These are all quantities that define the radial and the mass grid. Once they are defined they are constant and should not be changed. Additionally, the Keplerian frequency is located here.

### `Simulation.grid.A`

Surface area of the annulus a grid cell spans.

$A_i = \pi \left( r_{i+\frac{1}{2}}^2 - r_{i-\frac{1}{2}}^2 \right)$

### `Simulation.grid.Nr`

Number of radial grid cells.

In [66]:
sim.grid.Nr

100

### `Simulation.grid.OmegaK`

Keplerian frequency given by

$\Omega_\mathrm{K} = \sqrt{\frac{G\,M_*}{r^3}}$.

### `Simulation.grid.r`

Radial grid cell centers. The radial grid cell centers are exactly in the middle between the grid cell interfaces.

In [67]:
sim.grid.r == 0.5 * (sim.grid.ri[:-1] + sim.grid.ri[1:])

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]

### `Simulation.grid.ri`

Locations of the grid cell interfaces. `Simulation.grid.ri[0]` and `Simulation.grid.ri[-1]` are the inner and outer grid boundaries.

### Update order

The update order of the grid quantities in the standard model is set to

In [68]:
sim.grid.updateorder

['OmegaK']

## Star

In [69]:
sim.star

Group (Stellar quantities)
--------------------------
    L            : Field (Luminosity [erg/s])
    M            : Field (Mass [g])
    R            : Field (Radius [cm])
    T            : Field (Effective temperature [K])
  -----

### `Simulation.star.L`

Stellar luminosity given by

$L = 4\pi\,R_*^2\,\sigma_\mathrm{SB}\,T_*^4$

### `Simulation.star.M`

Stellar mass. Initially set to the value given in `Simulation.ini.star.M`, which corresponds to one Solar mass.

In [70]:
sim.star.M

1.988409870698051e+33

### `Simulation.star.R`

Stellar radius. Initially set to the value given in `Simulation.ini.star.R`, which corresponds to 2 Solar radii.

In [71]:
sim.star.R

139140000000.0

### `Simulation.star.T`

Stellar effective surface temperature. Initially set to the value given in `Simulation.ini.star.T`.

In [72]:
sim.star.T

5772.0

### Update order

The update order of the stellar quantities in the standard model is set to

In [73]:
sim.star.updateorder

['M', 'R', 'T', 'L']

## Time

`Simulation.t` is the current time and the integration variable. It starts at zero initially.

In [74]:
sim.t

0.0

Snapshots are written between $10^3$ years and $10^5$ years with 10 snapshots per time decade.

In [75]:
sim.t.snapshots

array([3.15576000e+10, 3.97286646e+10, 5.00154254e+10, 6.29656900e+10,
       7.92691073e+10, 9.97938935e+10, 1.25633068e+11, 1.58162662e+11,
       1.99114995e+11, 2.50670927e+11, 3.15576000e+11, 3.97286646e+11,
       5.00154254e+11, 6.29656900e+11, 7.92691073e+11, 9.97938935e+11,
       1.25633068e+12, 1.58162662e+12, 1.99114995e+12, 2.50670927e+12,
       3.15576000e+12])

The timestep is calculated from the dust and the gas source terms and the current values of the surface densities of dust and gas, while using a safety factor of `0.1`, which can be accessed as an attribute of the integration variable.

In [76]:
sim.t.cfl

0.1

## Integrator

The integrator that is used by default as two integration instructions. One for the gas and one for the dust.

In [77]:
sim.integrator

Integrator (Default integrator)

In [78]:
sim.integrator.instructions

[Instruction (Dust (state vector): implicit 1st-order direct solver),
 Instruction (H2: implicit 1st-order direct solver for tracers),
 Instruction (He: implicit 1st-order direct solver for tracers)]

The gas is integrated with an implicit first-order Euler integration scheme. The Jacobian is calculated from the parameters given in `Simulation.gas`. Parameters like gas velocities, fluxes and source terms are calculated once the new values of the gas surface density have been found.

Except for the values at the boundaries, the advective source terms are given by

In [79]:
sim.gas.Sigma.jacobian() @ sim.gas.Sigma

array([ 0.00000000e+00, -2.24399443e-11, -1.28157418e-11, -1.19319272e-11,
       -1.11071620e-11, -1.03375091e-11, -9.61929417e-12, -8.94908870e-12,
       -8.32369316e-12, -7.74012197e-12, -7.19558913e-12, -6.68749498e-12,
       -6.21341377e-12, -5.77108205e-12, -5.35838788e-12, -4.97336074e-12,
       -4.61416214e-12, -4.27907675e-12, -3.96650432e-12, -3.67495192e-12,
       -3.40302686e-12, -3.14943002e-12, -2.91294961e-12, -2.69245537e-12,
       -2.48689319e-12, -2.29527997e-12, -2.11669899e-12, -1.95029542e-12,
       -1.79527227e-12, -1.65088648e-12, -1.51644538e-12, -1.39130330e-12,
       -1.27485841e-12, -1.16654984e-12, -1.06585487e-12, -9.72286384e-13,
       -8.85390432e-13, -8.04743987e-13, -7.29952798e-13, -6.60649393e-13,
       -5.96491191e-13, -5.37158721e-13, -4.82353949e-13, -4.31798681e-13,
       -3.85233062e-13, -3.42414144e-13, -3.03114532e-13, -2.67121091e-13,
       -2.34233717e-13, -2.04264166e-13, -1.77034942e-13, -1.52378244e-13,
       -1.30134961e-13, -

The total source terms, i.e., including external sources and excluding the boundaries are given by

In [80]:
sim.gas.Sigma.jacobian() @ sim.gas.Sigma + sim.gas.S.ext

[ 0.00000000e+00 -2.24399443e-11 -1.28157418e-11 -1.19319272e-11
 -1.11071620e-11 -1.03375091e-11 -9.61929417e-12 -8.94908870e-12
 -8.32369316e-12 -7.74012197e-12 -7.19558913e-12 -6.68749498e-12
 -6.21341377e-12 -5.77108205e-12 -5.35838788e-12 -4.97336074e-12
 -4.61416214e-12 -4.27907675e-12 -3.96650432e-12 -3.67495192e-12
 -3.40302686e-12 -3.14943002e-12 -2.91294961e-12 -2.69245537e-12
 -2.48689319e-12 -2.29527997e-12 -2.11669899e-12 -1.95029542e-12
 -1.79527227e-12 -1.65088648e-12 -1.51644538e-12 -1.39130330e-12
 -1.27485841e-12 -1.16654984e-12 -1.06585487e-12 -9.72286384e-13
 -8.85390432e-13 -8.04743987e-13 -7.29952798e-13 -6.60649393e-13
 -5.96491191e-13 -5.37158721e-13 -4.82353949e-13 -4.31798681e-13
 -3.85233062e-13 -3.42414144e-13 -3.03114532e-13 -2.67121091e-13
 -2.34233717e-13 -2.04264166e-13 -1.77034942e-13 -1.52378244e-13
 -1.30134961e-13 -1.10153732e-13 -9.22900668e-14 -7.64055307e-14
 -6.23669975e-14 -5.00459841e-14 -3.93180669e-14 -3.00623929e-14
 -2.21612907e-14 -1.54999

The dust is integrated with an implicit first-order Euler integration scheme. The Jacobian is calculated from the parameters given in `Simulation.dust`. Parameters like gas velocities, fluxes and source terms are calculated once the new values of the gas surface density have been found.

Except for the values at the boundaries, the hydrodynamic and coagulation source terms are given by

In [81]:
(sim.dust.Sigma.jacobian() @ sim.dust.Sigma.ravel()).reshape(sim.dust.Sigma.shape)

array([[ 1.62742573e-10,  1.91382226e-10],
       [-8.62624259e-09,  8.62624259e-09],
       [-6.76005711e-09,  6.76005711e-09],
       [-5.29697964e-09,  5.29697964e-09],
       [-4.15004894e-09,  4.15004894e-09],
       [-3.25104399e-09,  3.25104399e-09],
       [-2.54644879e-09,  2.54644879e-09],
       [-1.99428545e-09,  1.99428545e-09],
       [-1.56162895e-09,  1.56162895e-09],
       [-1.22265665e-09,  1.22265665e-09],
       [-9.57117904e-10,  9.57117904e-10],
       [-7.49133018e-10,  7.49133018e-10],
       [-5.86250947e-10,  5.86250947e-10],
       [-4.58709924e-10,  4.58709924e-10],
       [-3.58857370e-10,  3.58857370e-10],
       [-2.80694808e-10,  2.80694808e-10],
       [-2.19520861e-10,  2.19520861e-10],
       [-1.71651199e-10,  1.71651199e-10],
       [-1.34198892e-10,  1.34198892e-10],
       [-1.04902120e-10,  1.04902120e-10],
       [-8.19890697e-11,  8.19890697e-11],
       [-6.40719780e-11,  6.40719780e-11],
       [-5.00640572e-11,  5.00640572e-11],
       [-3.

Note the `ravel()` and `reshape()` operations, since the Jacobian works on a one-dimensional state vector.

The total source terms except at the boundaries are given by

In [82]:
(sim.dust.Sigma.jacobian() @ sim.dust.Sigma.ravel()).reshape(sim.dust.Sigma.shape) + sim.dust.S.ext

[[ 1.62742573e-10  1.91382226e-10]
 [-8.62624259e-09  8.62624259e-09]
 [-6.76005711e-09  6.76005711e-09]
 [-5.29697964e-09  5.29697964e-09]
 [-4.15004894e-09  4.15004894e-09]
 [-3.25104399e-09  3.25104399e-09]
 [-2.54644879e-09  2.54644879e-09]
 [-1.99428545e-09  1.99428545e-09]
 [-1.56162895e-09  1.56162895e-09]
 [-1.22265665e-09  1.22265665e-09]
 [-9.57117904e-10  9.57117904e-10]
 [-7.49133018e-10  7.49133018e-10]
 [-5.86250947e-10  5.86250947e-10]
 [-4.58709924e-10  4.58709924e-10]
 [-3.58857370e-10  3.58857370e-10]
 [-2.80694808e-10  2.80694808e-10]
 [-2.19520861e-10  2.19520861e-10]
 [-1.71651199e-10  1.71651199e-10]
 [-1.34198892e-10  1.34198892e-10]
 [-1.04902120e-10  1.04902120e-10]
 [-8.19890697e-11  8.19890697e-11]
 [-6.40719780e-11  6.40719780e-11]
 [-5.00640572e-11  5.00640572e-11]
 [-3.91143568e-11  3.91143568e-11]
 [-3.05566996e-11  3.05566996e-11]
 [-2.38696529e-11  2.38696529e-11]
 [-1.86451555e-11  1.86451555e-11]
 [-1.45639294e-11  1.45639294e-11]
 [-1.13762138e-11  1

At the end of a successful integration step, the floor values and boundaries are enforced.

## Writer

`TwoPopPy` uses by default the `hdf5writer` of `simframe`.

In [83]:
sim.writer

Writer (HDF5 file format using h5py)
------------------------------------
    Data directory : data
    File names     : data/data0000.hdf5
    Overwrite      : False
    Dumping        : True
    Options        : {'com': 'lzf', 'comopts': None}
    Verbosity      : 1